In [1]:
import numpy as np
import pandas as pd

from datetime import date
from datetime import datetime

from sklearn.neighbors import KernelDensity

import seaborn as sns
import random as r

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
# For downsampling
from sklearn.utils import resample,shuffle
np.random.seed(2021)

In [2]:
df_companies = pd.read_csv("./data/companies.csv")
df_payments = pd.read_csv("./data/payments.csv")
df_physicians = pd.read_csv("./data/physicians.csv")

c:\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df_companies.rename({"State":"State_Comp","Country":"Country_Comp"}, axis='columns', inplace=True)
df_physicians.rename({"State":"State_Phys","Country":"Country_Phys"}, axis='columns', inplace=True)
pd.set_option('display.max_columns', None)


In [21]:
df_payments_test = df_payments

In [27]:
df_payments['Ownership_Interest']=0
for phys_id in df_payments.Physician_ID.unique():
    df_scrap=df_payments.query('Physician_ID==@phys_id and Ownership_Indicator=="Yes"')
    for comp in df_scrap.Company_ID.unique():
        df_scrap2=df_payments.query('Company_ID==@comp')
        for rec in df_scrap2.Record_ID.unique():
            df_payments['Ownership_Interest'].iloc[rec-1]=1

c:\python38\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
c:\python38\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
c:\python38\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, va

c:\python38\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
c:\python38\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
c:\python38\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, va

c:\python38\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
c:\python38\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
c:\python38\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, va

c:\python38\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
c:\python38\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
c:\python38\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, va

KeyboardInterrupt: 

In [29]:
df_scrap=df_payments.query('Ownership_Indicator=="Yes"')
df=df_scrap[['Company_ID','Physician_ID']]
df

,Company_ID,Physician_ID
32,1224,3935
42,707,1132
163,707,2256
168,707,2256
175,707,2256
...,...,...
1389314,1760,3051
1390881,2310,3969
1399823,2087,576
1402167,772,1034


In [30]:
df_comb=df.drop_duplicates()

In [32]:
df_payments['Ownership_Interest']=0
for i in range(df_payments.shape[0]):
    company=df_payments['Company_ID'].iloc[i]
    phys=df_payments['Physician_ID'].iloc[i]
    if(df_comb.query('Company_ID==@company and Physician_ID==@phys').shape[0]>0):
        df_payments['Ownership_Interest'].iloc[i]=1

In [33]:
#combine into one df
df = pd.merge(df_payments, df_physicians, left_on='Physician_ID', right_on='id', how='left').drop('id', axis=1)
df = pd.merge(df, df_companies, on='Company_ID', how='left')

In [38]:
df_payments.query('Ownership_Indicator=="Yes"')

,Record_ID,Physician_ID,Company_ID,Total_Amount_of_Payment_USDollars,Date,Number_of_Payments,Form_of_Payment_or_Transfer_of_Value,Nature_of_Payment_or_Transfer_of_Value,City_of_Travel,State_of_Travel,Country_of_Travel,Ownership_Indicator,Third_Party_Recipient,Charity,Third_Party_Covered,Contextual_Information,Related_Product_Indicator,Product_Code_1,Product_Code_2,Product_Code_3,Product_Type_1,Product_Type_2,Product_Type_3,Product_Name_1,Product_Name_2,Product_Name_3,Product_Category_1,Product_Category_2,Product_Category_3,Ownership_Interest
32,33,3935,1224,1200.00,08/01/2013,4,Cash or cash equivalent,Compensation for services other than consultin...,NaN,NaN,NaN,Yes,No Third Party Payment,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
42,43,1132,707,24.76,08/01/2013,4,Cash or cash equivalent,Compensation for services other than consultin...,NaN,NaN,NaN,Yes,Individual,No,Yes,Physician did not actually receive this paymen...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
163,164,2256,707,25.48,08/01/2013,4,Cash or cash equivalent,Compensation for services other than consultin...,NaN,NaN,NaN,Yes,Individual,No,Yes,Physician did not actually receive this paymen...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
168,169,2256,707,7.43,08/01/2013,4,Cash or cash equivalent,Compensation for services other than consultin...,NaN,NaN,NaN,Yes,No Third Party Payment,No,NaN,Physician did not actually receive this paymen...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
175,176,2256,707,87.35,08/01/2013,4,Cash or cash equivalent,Compensation for services other than consultin...,NaN,NaN,NaN,Yes,No Third Party Payment,No,NaN,Physician did not actually receive this paymen...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389314,1389315,3051,1760,5278.57,11/25/2019,1,Cash or cash equivalent,Royalty or License,NaN,NaN,NaN,Yes,No Third Party Payment,NaN,NaN,NaN,Yes,NaN,NaN,NaN,Device,NaN,NaN,NaN,NaN,NaN,Medical Implant,NaN,NaN,0
1390881,1390882,3969,2310,1797.00,12/02/2019,3,Cash or cash equivalent,Consulting Fee,NaN,NaN,NaN,Yes,No Third Party Payment,NaN,NaN,NaN,Yes,NaN,NaN,NaN,Device,NaN,NaN,Intellijoint HIP,NaN,NaN,Orthopedics,NaN,NaN,0
1399823,1399824,576,2087,169.50,12/18/2019,1,Cash or cash equivalent,Food and Beverage,NaN,NaN,NaN,Yes,No Third Party Payment,NaN,NaN,NaN,Yes,NaN,NaN,NaN,Device,NaN,NaN,SureClip and Traction Device,NaN,NaN,GI Disposables,NaN,NaN,0
1402167,1402168,1034,772,123202.50,12/31/2019,1,Stock,Consulting Fee,NaN,NaN,NaN,Yes,No Third Party Payment,NaN,NaN,"Consulting services including speaking, produc...",Yes,NaN,NaN,NaN,Device,Device,Device,Penumbra System,Penumbra SMART Coil,Penumbra Coil 400,Neurointervention,Neurointervention,Neurointervention,0


In [ ]:
df = df.astype(str)
for column in df.columns:
     if(df[column].dtype=='object'):
        df[column] = df[column].str.lower()
        
#Convert to Date
df['Date']=pd.to_datetime(df['Date'])

#type conv after str conv
df[['Company_ID','Number_of_Payments']]=df[['Company_ID','Number_of_Payments']].astype(int)
df['Total_Amount_of_Payment_USDollars']=df['Total_Amount_of_Payment_USDollars'].astype(float)

name_list = pd.unique(df[['Product_Name_1', 'Product_Name_2', 'Product_Name_3']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Product_Name_1'] = df['Product_Name_1'].apply(product_encode)
df['Product_Name_2'] = df['Product_Name_2'].apply(product_encode)
df['Product_Name_3'] = df['Product_Name_3'].apply(product_encode)

name_list = pd.unique(df[['Product_Code_1', 'Product_Code_2', 'Product_Code_3']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Product_Code_1'] = df['Product_Code_1'].apply(product_encode)
df['Product_Code_2'] = df['Product_Code_2'].apply(product_encode)
df['Product_Code_3'] = df['Product_Code_3'].apply(product_encode)

name_list = pd.unique(df[['Product_Category_1', 'Product_Category_2', 'Product_Category_3']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Product_Category_1'] = df['Product_Category_1'].apply(product_encode)
df['Product_Category_2'] = df['Product_Category_2'].apply(product_encode)
df['Product_Category_3'] = df['Product_Category_3'].apply(product_encode)

name_list = pd.unique(df[['Product_Type_1', 'Product_Type_2','Product_Type_3']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Product_Type_1'] = df['Product_Type_1'].apply(product_encode)
df['Product_Type_2'] = df['Product_Type_2'].apply(product_encode)
df['Product_Type_3'] = df['Product_Type_3'].apply(product_encode)


name_list = pd.unique(df[['License_State_1', 'License_State_2', 'License_State_3', 'License_State_4', 'License_State_5']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def license_encode(license_state):
    return name_list_dict.get(license_state)

df['License_State_1']=df['License_State_1'].apply(license_encode)
df['License_State_2']=df['License_State_2'].apply(license_encode)
df['License_State_3']=df['License_State_3'].apply(license_encode)
df['License_State_4']=df['License_State_4'].apply(license_encode)
df['License_State_5']=df['License_State_5'].apply(license_encode)

name_list = pd.unique(df[['Country_Comp', 'Country_Phys','Country_of_Travel']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Country_Comp'] = df['Country_Comp'].apply(product_encode)
df['Country_Phys'] = df['Country_Phys'].apply(product_encode)
df['Country_of_Travel'] = df['Country_of_Travel'].apply(product_encode)

name_list = pd.unique(df[['State_Comp', 'State_Phys', 'State_of_Travel']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['State_Comp'] = df['State_Comp'].apply(product_encode)
df['State_Phys'] = df['State_Phys'].apply(product_encode)
df['State_of_Travel'] = df['State_of_Travel'].apply(product_encode)

name_list = pd.unique(df[['City_of_Travel', 'City']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['City_of_Travel'] = df['City_of_Travel'].apply(product_encode)
df['City'] = df['City'].apply(product_encode)

df['Ownership_Indicator']=np.where((df.Ownership_Indicator == "yes"),1,df.Ownership_Indicator)
df['Ownership_Indicator']=np.where((df.Ownership_Indicator == "no"),0,df.Ownership_Indicator)

df['Date_year']=df['Date'].dt.year
df['Date_month']=df['Date'].dt.month
df['Date_day']=df['Date'].dt.day

columns_encode=['Nature_of_Payment_or_Transfer_of_Value','Form_of_Payment_or_Transfer_of_Value','Zipcode','Third_Party_Recipient', 'Charity', 'Third_Party_Covered','Contextual_Information', 'Related_Product_Indicator','Primary_Specialty']
for column in columns_encode:
    df[column]=df[column].astype('category')
    df[column]=df[column].cat.codes

print(df.dtypes)
df

In [23]:
print(df_physicians.head())
print(df_payments.head())


   id    set First_Name Middle_Name Last_Name Name_Suffix         City  \
0   1  train      LEIGH           B     HOPPE         NaN  FORT PIERCE   
1   2  train     STEVEN         NaN     FRANK         NaN       TOWSON   
2   3  train     THOMAS         NaN     COUCH         JR.         TROY   
3   4  train     CLAUDE         NaN        SU         NaN     SAVANNAH   
4   5  train     ROBERT           C   SCHENCK         NaN  ALBUQUERQUE   

  State_Phys     Zipcode   Country_Phys  Province  \
0         FL       34950  UNITED STATES       NaN   
1         MD  21204-6808  UNITED STATES       NaN   
2         NY  12180-2832  UNITED STATES       NaN   
3         GA  31419-1753  UNITED STATES       NaN   
4         NM  87131-0001  UNITED STATES       NaN   

                                   Primary_Specialty License_State_1  \
0  Allopathic & Osteopathic Physicians|Obstetrics...              FL   
1  Allopathic & Osteopathic Physicians|Anesthesio...              MD   
2  Podiatric Medicin

In [12]:
df_payments.Form_of_Payment_or_Transfer_of_Value.value_counts()

In-kind items and services                              1170221
Cash or cash equivalent                                  231967
Dividend, profit or other return on investment               38
Stock                                                        13
Any other ownership interest                                  4
Stock, stock option, or any other ownership interest          4
Stock option                                                  3
Name: Form_of_Payment_or_Transfer_of_Value, dtype: int64

In [4]:
def ownInterest(id):
    if(df_payments.query('Physician_ID==@id and Ownership_Indicator=="Yes"').shape[0]>0):
        return 1
    else:
        return 0
    
df_physicians['Ownership_Interest']=df_physicians['id'].apply(ownInterest)

In [8]:
df_physicians.query('set=="train"')['Ownership_Interest'].value_counts()

0    4700
1     300
Name: Ownership_Interest, dtype: int64

In [16]:
#drop Ownership Indicators=yes records
df_scrap=df_payments.query('Ownership_Indicator=="Yes"', engine='python')
df_payments=pd.concat([df_payments, df_scrap]).drop_duplicates(keep=False)


In [ ]:
df_payments['Date']=pd.to_datetime(df_payments['Date'])

